
<a href="https://colab.research.google.com/github/isikus/MachineScorning/blob/main/Нагенерировать%20машинного%20отупения%20на%20ruGPT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Нагенерировать машинного отупения на ruGPT3

In [ ]:
import os
import json

def is_empty(fpath, default_lines=[["Дополни эту фразу", "И эту фразу тоже дополни"], ["Первый пример запроса", "Из нескольких строк", "", "И второй пример", "Я тоже придумать смог"]]):
  if not os.path.isfile(fpath):
    return True
  if not (os.path.getsize(fpath) > 0):
    return True
  if default_lines:
    with open(fpath, "r") as infile:
      lines = infile.read().split("\n")
    for deflines in default_lines:
      if lines == deflines:
        return True
  return False
  

def get_lbc(model_meta):
  lbc = model_meta["line_break_character"]
  lbc = lbc.replace("\s", " ")
  return lbc

#@markdown Для начала выберите модель:

Модель = "\u041D\u0435\u0439\u0440\u043E\u0438\u043D\u0441\u0442\u0440\u0443\u043A\u0446\u0438\u0438 \u043D\u0430 \u0432\u0441\u0435 \u0441\u043B\u0443\u0447\u0430\u0438 \u0436\u0438\u0437\u043D\u0438"  #@param ["Нейроописания товаров", "Нейроинструкции на все случаи жизни", "Нейротексты «Короля и Шута»", "Нейроновости Северной Кореи", "Нейропоздравления с новым годом"]

desc_dict = {
    "Нейроописания товаров": "products_desc",
    "Нейроинструкции на все случаи жизни": "wikihowto",
    "Нейротексты «Короля и Шута»": "kish_verses",
    "Нейроновости Северной Кореи": "rukcna",
    "Нейропоздравления с новым годом": "greetings"
}

model = desc_dict[Модель]

!wget -q --show-progress https://storage.googleapis.com/ml-bucket-isikus/machine-scorning/{model}.tar.gz
!mkdir {model}
!tar -xzf {model}.tar.gz -C {model}

try:
  with open(model+"/meta.json", "r") as injson:
    model_meta = json.load(injson)
  print('\x1b[1m'+Модель + ": модель подготовлена"+'\x1b[0m')
except Exception as e:
  print('\x1b[1;31m'+"Что-то сломалось – модель не готова :("+'\x1b[0m')

wikihowto.tar.gz    100%[===================>] 443.70M   109MB/s    in 4.4s    
Нейроинструкции на все случаи жизни: модель подготовлена


Теперь запишите варианты начала сообщений, которые должна дополнить модель, сразу под текстом `%%writefile input.txt`, по 1 разному началу на строку:

In [ ]:
%%writefile input.txt
Дополни эту фразу
И эту фразу тоже дополни

Writing input.txt


**Важно:** если ваши запросы содержат переносы строк (например, это начало стихотворения), вместо предыдущей клетки, заполните клетку ниже, разбивая ваш ввод двумя переносами строк. Код ниже заменит переносы строк на специальный символ – это необходимо для корректной работы модели.

In [ ]:
%%writefile input_with_lines.txt
Первый пример запроса
Из нескольких строк

И второй пример
Я тоже придумать смог

Writing input_with_lines.txt


Если у вас совсем нет идей, запустите код ниже – он скачает примеры запросов по умолчанию для вашей модели:

In [ ]:
if is_empty("input.txt") and is_empty("input_with_lines.txt"):
  !cp {model}/example_input.txt input.txt
  print('\x1b[1;31m'+"Загружены примеры по умолчанию"+'\x1b[0m')

In [ ]:
#@markdown Наконец, укажите, сколько вариантов выдачи нужно сгенерировать модели на каждый из запросов (от 1 до 30):

Повторов = 4  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Это маловато. Установили значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Это многовато. Установили значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так. Установили значение " + str(takes) + ".")

## Подготавливаем среду для модели
Сами файлы скачаны, но нужно ещё выполнить код, чтобы подготовить модель к работе.

Проверяем, что мы подключены к видеокарте: команда в клетке ниже должна работать и показывать показывать примерно 12000-15000 Мб свободной памяти:

In [ ]:
!nvidia-smi

Sun Feb 14 11:24:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Устанавливаем нужные версии библиотек:

In [ ]:
!pip3 install -q urllib3==1.25.4 transformers==2.8.0

print('\n'+'\x1b[1m'+"Требуемые версии пакетов установлены"+'\x1b[0m')

     |████████████████████████████████| 133kB 14.8MB/s 
     |████████████████████████████████| 573kB 31.4MB/s 
     |████████████████████████████████| 133kB 59.2MB/s 
     |████████████████████████████████| 890kB 56.3MB/s 
     |████████████████████████████████| 1.2MB 53.1MB/s 
     |████████████████████████████████| 3.7MB 47.5MB/s 
     |████████████████████████████████| 7.2MB 58.7MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.

Требуемые версии пакетов установлены


Скачиваем файл с исходным кодом для работы модели:

In [ ]:
!wget -q --show-progress https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

print('\n'+'\x1b[1m'+"Код запуска модели загружен"+'\x1b[0m')

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

Код запуска модели загружен


Запускаем код, необходимый для преобразования файлов и отображения выдачи:

In [ ]:
import re
import os
import sys
import json
import random
from datetime import datetime

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

with open(model+"/meta.json", "r") as injson:
  model_meta = json.load(injson)

# converting inputs

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

print('\x1b[1m'+"Среда подготовлена"+'\x1b[0m')

Среда подготовлена


## Генерируем отупение

Теперь всё, наконец, готово к тому, чтобы модель сгенерировала нам выдачу. То, насколько серьёзный и осмысленный текст будет пытаться выдавать модель, можно регулировать по шкале **`Бредовость`**: здесь есть 5 степеней – от самой нормальной до самой странной (я предупреждал). Кроме этого, флажок **`Повторяться`** определяет, как будет работать модель: если галочку оставить, то первый результат будет очень похож не те, что есть в канале, а последующие будут с ним совпадать. А если снять галочку, то выдача модели каждый раз будет новая. Удачи!

In [ ]:
#@markdown ### Начать генерацию!

Бредовость = 75 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

stup_cases = {
    0: {"k": 5, "p": 0.95, "temp": 1.0},
    25: {"k": 5, "p": 0.93, "temp": 2.0},
    50: {"k": 6, "p": 0.91, "temp": 3.0},
    75: {"k": 6, "p": 0.9, "temp": 4.0},
    100: {"k": 7, "p": 0.88, "temp": 5.0}
}

Повторяться = True   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
for resp in resps:
  pt, rs, _ = re.findall(re.compile(repr(prompt)[1:-1] + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>> ruGPT:")] + "<br>")

Ключ случайного значения: **42**

**А как какать?**

Сделайте глубокий вдох. Поддерживаем водно-солевой баланс организма, особенно если вам приходится есть в одно-два блюда каждый день. Попробуйте позыв «согреться»: Попробуйте позыв «лечь поспать»; это поможет вам снять мышечные боли; если это не подействовало, то вы должны встать с постели в тот период, который вам не подходит; вы не обязаны лежать в одной кровати. Попробуйте позывы для расслабления; это будет похоже на позывы на расслабление после еды или после того как вы ее сделали<br>

**А как какать?**

Сосчитай до трех. Разрежь свой палец пополам. Сожмите большой и средний кулак. Встаньте прямо на ноги. Поднимите средний кулак выше уровня головы и поднимите большой кулак вверх. Повращением указательным пальцем сделайте два больших шага вперед (передний и задний план соответственно). Вытянитесь вперед и назад, а также согнитесь в три оборота в одну линию. Поднимитесь в полный рост на носочке и поднимитесь на верхнюю точку на передней части бедра (с согнутой ногой и вперед). Выпрямите верхнюю и заднюю часть туловища и повторите шаги с обеих сторон от себя до конца бедра и вверх (на бедру). Выпрямите руки по швам. Сдвиньте большой и средний пальцы в положение «назад». Встаньте в полный рост перед большим и средним пальцем, чтобы поднять большой палец. Выпрямите левую руку. Переведите указательным пальцем в положение на «передний план» (если вы не можете перевести палец в исходное положение) (это делается, чтобы не терять сознание от страха и боли). Выпрямите верхнюю часть спины (впрямоугольником вниз, с правой стороны, если вам больше 18 лет, и ваша спина больше по объему и ширине) (это упражнение делается, как показано ниже, в качестве альтернативного метода для уменьшения объема тела и поднятию верхней и боковой частей корпуса). Переведите свой палец назад (на верхушку, если он находится между большим и средним) и в положении «задний план», чтобы поднять средний палец, а затем в положение для увеличения верхней руки (если у вас нет большого размера). Сдвиньте мизинец в исходное положение и поднимитесь на носочку. В положении «сгиб вперед», опускаясь в положение «согнута» (как показано ниже) и выпрямляйтесь<br>

...

Утки — хищные птицы семейства утицовых; в дикой природе встречаются в разных местах по всему миру. <br>THIS MEME WORLD OCKIY SYSTEMS: [1] <br>WARNING OWIES WITH A WH.OWIETHEY SYSTEМ (2:00 PHP, C:0001:1). [1] <br>WORLD WH.LOG SWITH WATSH (4.2:30 PMP:1). [1]. [1] Утки, как и птицы-слоны:  Утка не должна прыгать на человека. <br>WH.showWARDS.create.add(document.getElementByImg('img_d5cdc5a7').v) <br> <br>WH.show.add('img_e9a6bbd6') <br> <br> Если хотите посмотреть, сколько утят у вас уже, вы можете найти утку на ферме<br>

**В чём разница между уткой?**

Определение утки по размерам тела:  Определение размера утки по размеру яйца <br> Оцените размеры яиц: Найдем размеры головы: Узнаем размеры уток по форме тела, чтобы определить размер утки по форме тела и весу: Сравним вес яиц:  Определение размеров яиц по размерам:  Узнаваем, в чем заключается разница размеров яиц:  Узнайте о том, что такое яйца, когда и как можно их есть и хранить. Обратите внимание, какие бывают яйца и какие виды уток бывают.  Определем размер головы (по размерам тела и веса)<br>

**В чём разница между уткой?**

Определитесь с тем, что означает утенок. Выберите птицу или самца для вылупливания и размножение. Найдите утку. Наймите хорошего профессионала по разведению утят и утку, который сможет обеспечить надлежащую среду обитания. Выберите уток. Определим размер утка, на котором вы хотите его вылупиться и на кого он должен быть рассчитан. Определите, сколько уток может нестись. Выберите подходящее гнездо, чтобы разместить утку. Определим количество самца и самка, если хотите получить уток, которые не имеют детеночка. Примите во внимание количество инфицированных утят и количество самцов и самок утят в вашем регионе. Выберите подходящую пару утков. Подготовку утки следует начать сразу же, как утки родятся. Выберите подходящее для уток гнездо. Приберите утку. Найдя птицу в гнездовом комоде и в подходящих местах, начните готовить утку и выталкивать утку, когда утка достигнет возраста 1,5–2,5 лет, когда ее вес увеличится примерно до 2 килограмм за один раз и до 4 килограмм за каждые 4 килограмма. <finished>end><br<br>

## Больше отупения!

Хотите попробовать другие фразы? Тогда впишите их в клетку под этим текстом, как до этого, и запустите код ниже. Не забудьте поменять `%%writefile input.txt` на `%%writefile input_with_lines.txt`, если ваши запросы содержат переносы строк!

In [32]:
%%writefile input.txt
Как быть, если вокруг такое?
Как поскорее пережить зиму?
Как правильно провести 14 февраля?

Overwriting input.txt


In [ ]:
#@markdown ### Генерируем новую выдачу

Повторов = 5  #@param {"type": "integer"}
takes = Повторов

if takes < 1:
  takes = 1
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if takes > 30:
  takes = 30
  print("Маловато повторов. Установлено значение " + str(takes) + ".")
if type(takes) != int:
  takes = 4
  print("Что-то пошло не так с количеством повторов. Установлено значение " + str(takes) + ".")

s = ""

if not is_empty("input_with_lines.txt"):
  with open("input_with_lines.txt", "r", encoding="utf-8") as intxt:
    prompts = [get_lbc(model_meta).join([sp for sp in p.split("\n") if sp]) for p in intxt.read().split("\n\n") if p]
else:
  with open("input.txt", "r", encoding="utf-8") as intxt:
    prompts = [p for p in intxt.read().split("\n") if p]

for prompt in prompts:
  s += (model_meta['prompt'] % prompt + '\n') * takes

with open("rawi.txt", "w", encoding="utf-8") as rawo:
  rawo.write(s)

Бредовость = 75 #@param {type: "slider", min: 0, max: 100, step: 25}
stlv = Бредовость

Повторяться = False   # @param {"type": "boolean"}
add_randomness = not Повторяться

rs = random.choice(list(range(0, 10000))) if add_randomness else 42
printmd("Ключ случайного значения: **"+str(rs)+"**")

responses = !python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path={model} \
    --k={str(stup_cases[stlv]["k"])} \
    --p={str(stup_cases[stlv]["p"])} \
    --seed={str(rs)} \
    --temperature={str(stup_cases[stlv]["temp"])} \
    --repetition_penalty=1.0 \
    --stop_token="{model_meta['sequence_end_character']}" \
    --length=500 < rawi.txt 2>/dev/null

responses = "\n".join(list(responses))

lbc = model_meta["line_break_character"]
ssc = model_meta["sequence_start_character"]
sec = model_meta["sequence_end_character"]
prompt = re.sub(r"%[a-z]+", "(.*?)", model_meta["prompt"])

resps = [r for r in responses.split("Context >>> ruGPT:\n") if r]
for resp in resps:
  pt, rs, _ = re.findall(re.compile(repr(prompt)[1:-1] + r"\n?(.*?)(" + ssc + "|$)", flags=re.DOTALL), resp)[0]
  pt = "<br>".join("**" + p + "**" for p in re.split(lbc, pt) if p)
  printmd(pt)
  rs = re.sub(lbc, "<br>", rs).strip()
  rs = re.sub(r"([^\s])<br>([^\s])", r"\1 <br> \2", rs)
  printmd(rs[:rs.find("Context >>> ruGPT:")] + "<br>")

Ключ случайного значения: **1425**

Context >>> 

На этом всё. Получайте удовольствие (надеюсь)!